In [64]:
import requests
import pg8000

In [65]:

r = requests.get('http://api.geonames.org/search?featureClass=P&country=FR&maxRows=10&orderby=population&type=json&username=yoannbouge'); 

result = r.json()['geonames']

In [66]:
resu = list(map(lambda x: {'geonameId': x['geonameId'], 'name': x[
            'name'], 'geometry': (x['lng']) + ' ' + x['lat']}, result))

In [67]:
conn = pg8000.connect(user = "postgres", host = "localhost", port = 5432,
                              database = "postgres", password = "password")

In [68]:

for city in resu:
    conn.run("INSERT INTO public.villes(geonameid, geom, name) Values({}, {}, '{}');".format(
        city['geonameId'], "ST_GeomFromText('POINT({})')".format(city['geometry']), city['name']))
conn.commit()